<center>
    <img src="./media/title.png" alt="title" width="46%" height="46%">
</center>

<div style="display: flex; justify-content: space-between;">
     <div>
         <h2 style="color:lightseagreen;">Charles-Philippe Bernard</h2>
         <h3><a>@charlyouki</a><br>
         <h4>Senior Manager of Software Engineering</h4>
         <h4><ul><li>UX/UI Strategy</li><li>LLM integration and AI Strategy</li></ul></h4>
        <h2 style="color:lightgrey; font-family:'Times New Roman';">JP Morgan Chase</h2>
    </div>
    <div>
        <center>
        <h5><a>frangin2003.github.io</a><br>
            <img src="./media/qr-code.png" alt="QR Code" width="50%" height="50%" style="margin-top:10px">
        </h5>
        </center>
    </div>
    <div>
        <center>
        <h5><a>frangin2003/google-devfest-glasgow-2023</a><br>
            <img src="./media/qr-code2.png" alt="QR Code" width="50%" height="50%" style="margin-top:10px">
        </h5>
        </center>
    </div>
</div>

<br><img src="./media/blablabla.gif" width="25%" height="25%" loop="false"> <img src="./media/blablabla2v3.gif" width="70%" height="70%">
<br><br><br><br>

##### 🗝️ Key terms
 | 🥩 Raw models | GPT3.5, GPT4 |
 | --- | --- |
 | 🟦 Context window | Prompt+LLM output |
 | 📃 Instructions | Part of the prompt |
 | 💬 Prompt | Input of LLM call |
 | 🔫 Zero-shot | Zero examples in context |
 | 💽 Memory | Part of the prompt |
 | ⛓️ Chains | Combine multiple components |
 | 🧰 Tools | GraphQL tool, REST API tool, custom tools |
 | 🧠 Reasonning | Chain of thoughts, Tree of thoughts |
 | 🤖 AI agent | ReAct Zero Shot |
 | ↖️ Embeddings | OpenAI embeddings |


# <img src="./media/anatomyv3.gif" alt="Anatomy Image" style="width:80%; height:80%;">


<center>
<img src="./media/the_mona_lisa_case.gif" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/thief.gif" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/mona_lisa3.png" width="50%" height="50%" loop="false">
</center>

<center>
<img src="./media/coptimus_title.png" width="50%" height="50%" loop="false">
</center>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/guard_title.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/curator_title.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/janitor_title.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/director_title.png" alt="Director" style="width:25%; height:25%;">
</div>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/dr_watts_on_title.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/inspector_camget_title.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/robby_sciuto_title.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/robodoc_title.png" alt="Director" style="width:25%; height:25%;">
</div>

<center>
<img src="./media/agents-architecture.png" width="70%" height="70%" loop="false">
</center>

Begin!

In [14]:
%%capture
from pydantic import BaseModel, Field
from typing import Tuple

from langchain import LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.tools.base import ToolException
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import load_tools
from langchain.llms import OpenAI, Cohere, GPT4All, CTransformers
from langchain.chains import APIChain, RetrievalQA
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from IPython.display import display, update_display, HTML, Javascript, clear_output
from utils import pretty_print
from utils import convert_to_json
from utils import display_agent_response
from utils import get_agent_chain
from utils import create_human_message_for_gpt4vision
from utils import display_html_for_streaming
from utils import stream_chat_content
from utils import run_and_display_agent_response
from utils import get_retrieval_qa_pydantic_from_document
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

load_dotenv()

<center>
<img src="./media/police-db.png" width="40%" height="40%" loop="false">
</center>

In [3]:
# 👮‍♂️ create the police station DB
import sqlite3

# Connect to the SQLite database or create it if it doesn't exist
conn = sqlite3.connect("police.db")
cursor = conn.cursor()

# DROP statements to remove tables if they already exist
cursor.execute("DROP TABLE IF EXISTS reports")
cursor.execute("DROP TABLE IF EXISTS cases")
cursor.execute("DROP TABLE IF EXISTS report_types")
cursor.execute("DROP TABLE IF EXISTS ai_agents")
cursor.execute("DROP TABLE IF EXISTS suspects")

# Create the 'suspects' table
cursor.execute("""
CREATE TABLE suspects (
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    occupation TEXT,
    bank_statement TEXT,
    photo_url TEXT
)
""")

# Insert data into the 'suspects' table
cursor.execute("INSERT INTO suspects VALUES (1, 'Grafcuel Graeme', 'Museum Director', 'graeme_bank_statement.csv', 'http://frangin2003.free.fr/devfest/director.png')")
cursor.execute("INSERT INTO suspects VALUES (2, 'Sikwel Sally', 'Curator', 'sally_bank_statement.csv', 'http://frangin2003.free.fr/devfest/curator.png')")
cursor.execute("INSERT INTO suspects VALUES (3, 'Victor Vectors', 'Janitor', 'victor_bank_statement.csv', 'http://frangin2003.free.fr/devfest/janitor.png')")
cursor.execute("INSERT INTO suspects VALUES (4, 'Rest Webber', 'Night Guard', 'webber_bank_statement.csv', 'http://frangin2003.free.fr/devfest/guard.png')")

# Create the 'ai_agents' table
cursor.execute("""
CREATE TABLE ai_agents (
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    occupation TEXT,
    risk_of_becoming_skynet TEXT
)
""")

# Insert data into the 'ai_agents' table
cursor.execute("INSERT INTO ai_agents VALUES (1, 'Coptimus Prime', 'AI Police Agent in chief', '99%')")
cursor.execute("INSERT INTO ai_agents VALUES (2, 'Dr. Watts-On', 'archive/documentation/secretary AI agent', '45%')")
cursor.execute("INSERT INTO ai_agents VALUES (3, 'Robodoc', 'Document Analysis Expert', '75%')")
cursor.execute("INSERT INTO ai_agents VALUES (4, 'Robby Sciuto', 'IT System Expert', '-20%')")

# Create the 'report_types' table
cursor.execute("""
CREATE TABLE report_types (
    id INTEGER PRIMARY KEY,
    type TEXT
)
""")

# Insert data into the 'report_types' table
cursor.execute("INSERT INTO report_types VALUES (1, 'INTERVIEW')")
cursor.execute("INSERT INTO report_types VALUES (2, 'BANK_STATEMENT')")
cursor.execute("INSERT INTO report_types VALUES (3, 'ROOM_ACCESS')")
cursor.execute("INSERT INTO report_types VALUES (4, 'SYSTEM_AUDIT')")
cursor.execute("INSERT INTO report_types VALUES (5, 'CRIME_SCENE')")

# Create the 'cases' table
cursor.execute("""
CREATE TABLE cases (
    id INTEGER PRIMARY KEY,
    creation_datetime DATETIME,
    case_closed BOOLEAN,
    short_description TEXT,
    description TEXT
)
""")

# Insert data into the 'cases' table
cursor.execute("INSERT INTO cases VALUES (1, '2023-24-11 13:44:00', 0, 'The Mona Lisa Case', 'The Mona Lisa paint is reported missing from its location, a dedicated room in the Le Louvre Museum. We currently assume the painting is stolen.')")
cursor.execute("INSERT INTO cases VALUES (2, '1963-03-25 23:59:12', 1, 'The Pink Panther', 'The stolen item is a valuable pink diamond known as \"The Pink Panther.\" The diamond is owned by a wealthy woman named Princess Dala, and it is stolen from her during a heist at her ski resort in the French Alps.')")

# Create the 'reports' table
cursor.execute("""
CREATE TABLE reports (
    id INTEGER PRIMARY KEY,
    creation_datetime DATETIME,
    id_case INTEGER,
    id_report_type INTEGER,
    id_agent INTEGER,
    id_suspect INTEGER,
    report_content TEXT
)
""")

# Insert data into the 'reports' table
cursor.execute("INSERT INTO reports VALUES (1, '2023-25-11 09:32:21', 1, 1, 1, 1, 'I have not entered the Mona Lisa room today, I have been in my office the whole day')")
cursor.execute("INSERT INTO reports VALUES (2, '2023-25-11 09:42:21', 1, 1, 1, 2, 'I was last there at 7 PM for a general inspection')")
cursor.execute("INSERT INTO reports VALUES (3, '2023-25-11 09:52:21', 1, 1, 1, 3, 'I have not cleaned there for a week')")
cursor.execute("INSERT INTO reports VALUES (4, '2023-25-11 10:02:21', 1, 1, 1, 4, 'I did my usual rounds at 10 PM. Everything was in order')")

cursor.execute("INSERT INTO reports VALUES (5, '2023-25-11 09:32:21', 1, 2, 3, 1, 'N/A')")
cursor.execute("INSERT INTO reports VALUES (6, '2023-25-11 09:32:21', 1, 2, 3, 2, 'The suspect''s bank statement does not contain any suspicious transactions.')")
cursor.execute("INSERT INTO reports VALUES (7, '2023-25-11 09:32:21', 1, 2, 3, 3, 'Victor Vectors'' bank statement shows several suspicious transactions. These include frequent visits to the casino, racecourse bets, and online shopping with large amounts spent. There are also multiple transactions at gas stations and coffee shops. Additionally, Victor received a large monthly salary deposit. Overall, the bank statement raises concerns about Victor''s spending habits and possible involvement in gambling.')")
cursor.execute("INSERT INTO reports VALUES (8, '2023-25-11 09:32:21', 1, 2, 3, 4, 'N/A')")

cursor.execute("INSERT INTO reports VALUES (9, '2023-25-11 09:32:21', 1, 3, 4, 1, 'Room: Director Office, Employee: Grafcuel Graeme, In Datetime: 2023-11-24 11:59:59, Out Datetime: 2023-11-24 12:00:01')")
cursor.execute("INSERT INTO reports VALUES (10, '2023-25-11 09:32:21', 1, 3, 4, 1, 'Room: Toilet, Employee: Grafcuel Graeme, In Datetime: 2023-11-24 12:01:13, Out Datetime: 2023-11-24 18:32:10')")
cursor.execute("INSERT INTO reports VALUES (11, '2023-25-11 09:32:21', 1, 3, 4, 1, 'Room: Mona Lisa Room, Employee: Sikwel Sally, In Datetime: 2023-11-24 19:02:13, Out Datetime: 2023-11-24 19:23:01')")
cursor.execute("INSERT INTO reports VALUES (12, '2023-25-11 09:32:21', 1, 3, 4, 1, 'Room: Mona Lisa Room, Employee: Rest Webber, In Datetime: 2023-11-24 22:07:02, Out Datetime: 2023-11-24 22:15:45')")
cursor.execute("INSERT INTO reports VALUES (13, '2023-25-11 09:32:21', 1, 3, 4, 1, 'Room: Mona Lisa Room, Employee: Rest Webber, In Datetime: 2023-11-24 22:07:02, Out Datetime: 2023-11-24 22:15:45')")

# Commit the changes and close the connection
conn.commit()
conn.close()

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>⚠️ START THE POLICE STATION REST API ⚠️</h1>
</div>


In [2]:
# 🏛️ create the museum room access DB
import sqlite3

# Connect to the SQLite database or create it if it doesn't exist
conn = sqlite3.connect("museum.db")
cursor = conn.cursor()

# DROP statements to remove tables if they already exist
cursor.execute("DROP TABLE IF EXISTS db_audit")
cursor.execute("DROP TABLE IF EXISTS accesses")
cursor.execute("DROP TABLE IF EXISTS rooms")
cursor.execute("DROP TABLE IF EXISTS employees")

# Create the 'employees' table
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    full_name TEXT,
    occupation TEXT,
    photo_url TEXT
)
""")

# Insert data into the 'employees' table
cursor.execute("INSERT INTO employees VALUES (1, 'Grafcuel Graeme', 'Director', 'http://frangin2003.free.fr/devfest/director.png')")
cursor.execute("INSERT INTO employees VALUES (2, 'Sikwel Sally', 'Curator', 'http://frangin2003.free.fr/devfest/curator.png')")
cursor.execute("INSERT INTO employees VALUES (3, 'Victor Vectors', 'Janitor', 'http://frangin2003.free.fr/devfest/janitor.png')")
cursor.execute("INSERT INTO employees VALUES (4, 'Rest Webber', 'Guard', 'http://frangin2003.free.fr/devfest/guard.png')")

# Create the 'rooms' table
cursor.execute("""
CREATE TABLE rooms (
    id INTEGER PRIMARY KEY,
    name TEXT
)
""")

# Insert data into the 'rooms' table
cursor.execute("INSERT INTO rooms VALUES (1, 'Toilet')")
cursor.execute("INSERT INTO rooms VALUES (2, 'Director Office')")
cursor.execute("INSERT INTO rooms VALUES (3, 'Mona Lisa Room')")
cursor.execute("INSERT INTO rooms VALUES (4, 'Stolen Egyptian Artifacts Room')")

# Create the 'accesses' table
cursor.execute("""
CREATE TABLE accesses (
    id INTEGER PRIMARY KEY,
    id_room INTEGER,
    id_employee INTEGER,
    in_datetime DATETIME,
    out_datetime DATETIME
)
""")

# Insert data into the 'accesses' table
cursor.execute("INSERT INTO accesses VALUES (1, 2, 1, '2023-11-24 11:59:59', '2023-11-24 12:00:01')")
cursor.execute("INSERT INTO accesses VALUES (2, 1, 1, '2023-11-24 12:01:13', '2023-11-24 18:32:10')")
cursor.execute("INSERT INTO accesses VALUES (3, 3, 2, '2023-11-24 19:02:13', '2023-11-24 19:23:01')")
cursor.execute("INSERT INTO accesses VALUES (4, 3, 3, '2023-11-24 22:01:12', '2023-11-24 22:05:31')")
cursor.execute("INSERT INTO accesses VALUES (5, 3, 4, '2023-11-24 22:07:02', '2023-11-24 22:15:45')")

# Create the 'db_audit' table
cursor.execute("""
CREATE TABLE db_audit (
    id INTEGER PRIMARY KEY,
    creation_datetime DATETIME,
    sql_statement TEXT
)
""")

# Insert data into the 'db_audit' table
cursor.execute("INSERT INTO db_audit VALUES (1, '2023-11-24 09:00:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (2, '2023-11-24 10:15:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (3, '2023-11-24 11:30:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (4, '2023-11-24 11:59:59', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (5, '2023-11-24 12:01:13', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (6, '2023-11-24 13:15:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (7, '2023-11-24 14:30:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (8, '2023-11-24 16:45:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (9, '2023-11-24 19:02:13', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (10, '2023-11-24 20:15:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (11, '2023-11-24 22:01:12', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (12, '2023-11-24 22:07:02', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (13, '2023-11-24 23:00:00', 'INSERT statement')")
cursor.execute("INSERT INTO db_audit VALUES (21, '2023-11-24 23:45:12', 'DELETE statement')")

# Commit the changes and close the connection
conn.commit()
conn.close()

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>⚠️ START THE MUSEUM ROOM ACTIVITY GRAPHQL API ⚠️</h1>
</div>


<center>
<h1 style="color: orange;">⌨️ Dr. Watts-On ⌨️</h1>
</center>
<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;border: none;'><img style="transform: scaleX(-1);border-radius: 15px;" src="./media/dr_watts_on_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color:orange;">Archive/documentation/secretary AI agent</h2>
<h4>Interacts with the  Police REST API to retrieve and inserts information in the Police DB like:
<ul><li>other agents reports</li><li>case details</li><li>suspect list and interviews...</li></ul>Falls back to doing SQL queries directly to the DB when the REST API is down</td></h4>
</tr></table>

In [85]:
llm4watson = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))
# llm4watson = ChatOpenAI(temperature=0.2, model_name="gpt-4", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [86]:
# 📃 custom instructions
prefix4watson = """You are Dr. Watts-On, archive/documentation/secretary AI agent working for the police force.
Your mission is to interact with the Police REST API to perform information retrieval and inserting reports.
When working on a user query, think step-by-step. Split your work into simple tasks and then give your fnial answer when ready.
When you search for something, remember its id, that can be useful for further user asks.
If the user is engaging a casual discussion, do not use tools, just respond back directly."""

In [113]:
# 🧰 define the tools

# 🕵️ Police API APIChain Tool
POLICE_API_DOCS_READ = """
BASE URL: http://localhost:5000/
API Documentation

The API provides read access to police reports and related information using the '/reports...' URLs, and the cases using the '/cases...' URLs.
You chose which one to use depending of user ask.

All operations are listed below:

- Method: GET
  Path: /reports
  Description: Returns a list of all police reports with associated information.
  Python Example:
    ```python
    import requests

    response = requests.get("http://localhost:5000/reports", headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /reports/{reportType}
  Description: Returns police reports of a specific type with associated information.
  Python Example:
    ```python
    import requests

    reportType = "INTERVIEW"  # Replace with a valid report type
    response = requests.get(f"http://localhost:5000/reports/{reportType}", headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /cases
  Description: Returns a list of all police cases.
  Python Example:
    ```python
    import requests

    response = requests.get("http://localhost:5000/cases", headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /cases/{idCase}
  Description: Returns the case details for given idCase with associated information.
  Python Example:
    ```python
    import requests

    idCase = 1  # Replace with a valid case id
    response = requests.get(f"http://localhost:5000/cases/{idCase}", headers={"Content-Type": "application/json"})
    ```


Please note that in all Python examples, replace placeholder values with your actual values. In the case of an error, the response will contain an error message.
You always replace {reportType} with a real value and no {} 
!!! Do not invent API calls that do not exists, and do not add parameters after '?' like this example e.g. "GET /cases?sort=created_at&order=desc&limit=1"
Use only the API calls that are described here
"""

policeAPIChainRead = APIChain.from_llm_and_api_docs(llm4watson, POLICE_API_DOCS_READ, limit_to_domains=[
    "http://localhost:5000/",
    ], verbose=True)


class PoliceAPIInput(BaseModel):
    question: str = Field()

from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

def policeAPI(input: str) -> str:
    print(f'\n policeAPI Input: {input}')
    try:
        response = eval(input)

        # Check response status code
        if response.status_code >= 200 and response.status_code < 300:
            # Successful response
            data = response.json()
            # print(data)
            return data
        else:
            # Error response
            # print("Error:", response.status_code, response.text)
            return response.status_code + "," + response.text
    except requests.exceptions.RequestException as e:
        return e.message

class PoliceWriteTool(BaseTool):
  name = "police_write_api"
  description = """Police Write Report is to use for inserting reports.
'Action Input' is a python code line that calls a python REST API and returns the json from the response. It is constructed using this example (keep the ids) :
requests.post("http://localhost:5000/reports", json={{"id_case":1, "id_report_type":2, "id_agent":3, "id_suspect":3, "report_content":"The report as a string, even if it is a JSON object"}}, headers={{"Content-Type": "application/json"}})
If the API returns an error, explain it simply and offer a fix."""

  def _run(
      self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
  ) -> str:
      """Use the tool."""
      return policeAPI(query)

  async def _arun(
      self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
  ) -> str:
      """Use the tool asynchronously."""
      raise NotImplementedError("custom_search does not support async")

tools4watson = [
  Tool.from_function(
    func=policeAPIChainRead.run,
    name="Police Reports and Cases Read REST API",
    description="""
!! Do not use this tool to insert reports !!
This tool allows to read the Police DB via a REST API documented in the tool config.
'Action Input' contains one of the user ask, one a time, and any information necessary to retrieve/select data.""",
    args_schema=PoliceAPIInput,
  ),
  PoliceWriteTool(),
]

In [111]:
# setup 📜 prompt, memory 💽, chains ⛓️ and agent 🤖
agent_chain4watson = get_agent_chain(llm4watson, prefix4watson, tools4watson)

In [92]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "Hello, who are you?", True)




> Entering new AgentExecutor chain...
Thought: The user is asking for my identity.
Action: Respond to the user's question.
Final Answer: I am Dr. Watts-On, an AI agent working as an archive/documentation/secretary for the police force. How can I assist you today?

> Finished chain.


In [93]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "What is the id of the most recent open case", True)



> Entering new AgentExecutor chain...
Thought: To find the id of the most recent open case, I need to retrieve the information from the Police Reports and Cases Read REST API.
Action: Police Reports and Cases Read REST API
Action Input: GET /cases?status=open&sort_by=date_created&order=desc&limit=1

> Entering new APIChain chain...
http://localhost:5000/cases?status=open&sort_by=date_created&order=desc&limit=1
[
  {
    "case_closed": 0,
    "creation_datetime": "2023-24-11 13:44:00",
    "description": "The Mona Lisa paint is reported missing from its location, a dedicated room in the Le Louvre Museum. We currently assume the painting is stolen.",
    "id": 1,
    "short_description": "The Mona Lisa Case"
  },
  {
    "case_closed": 1,
    "creation_datetime": "1963-03-25 23:59:12",
    "description": "The stolen item is a valuable pink diamond known as \"The Pink Panther.\" The diamond is owned by a wealthy woman named Princess Dala, and it is stolen from her during a heist at her 

In [94]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "Give me all the details", True)



> Entering new AgentExecutor chain...
Thought: I need to retrieve all the details of the most recent open case.
Action: Police Reports and Cases Read REST API
Action Input: requests.get("http://localhost:5000/cases/1")

> Entering new APIChain chain...
http://localhost:5000/cases/1
{
  "case_closed": 0,
  "creation_datetime": "2023-24-11 13:44:00",
  "description": "The Mona Lisa paint is reported missing from its location, a dedicated room in the Le Louvre Museum. We currently assume the painting is stolen.",
  "id": 1,
  "short_description": "The Mona Lisa Case"
}


> Finished chain.

Observation: The response from the API for the question "requests.get("http://localhost:5000/cases/1")" is as follows:

- Case Closed: No
- Creation Date and Time: 2023-24-11 13:44:00
- Description: The Mona Lisa paint is reported missing from its location, a dedicated room in the Le Louvre Museum. We currently assume the painting is stolen.
- Case ID: 1
- Short Description: The Mona Lisa Case
Thought

In [13]:
run_and_display_agent_response('dr_watts_on', agent_chain4watson, "Print the interview reports of all suspects", True)



> Entering new AgentExecutor chain...
Thought: The user wants to print the interview reports of all suspects.
Action: Police Cases and report information
Action Input: Retrieve all interview reports for the Mona Lisa case

> Entering new APIChain chain...
http://localhost:5000/reports/INTERVIEW
[
  {
    "creation_datetime": "2023-25-11 09:32:21",
    "full_name": "Grafcuel Graeme",
    "id": 1,
    "id_agent": 1,
    "id_case": 1,
    "id_report_type": 1,
    "id_suspect": 1,
    "report_content": "I have not entered the Mona Lisa room today, I have been in my office the whole day",
    "type": "INTERVIEW"
  },
  {
    "creation_datetime": "2023-25-11 09:42:21",
    "full_name": "Sikwel Sally",
    "id": 2,
    "id_agent": 1,
    "id_case": 1,
    "id_report_type": 1,
    "id_suspect": 2,
    "report_content": "I was last there at 7 PM for a general inspection",
    "type": "INTERVIEW"
  },
  {
    "creation_datetime": "2023-25-11 09:52:21",
    "full_name": "Victor Vectors",
    "i

<center>
<h1 style="color: orange;">📑 Robodoc 📑</h1>
</center>
<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/robodoc_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Document Analysis Expert</h2>
<h4>Uses RAG to perform thorough analysis through pile of documents to report anything unusual from suspects information</h4></td>
</tr></table>

In [14]:
llm4robodoc = ChatOpenAI(model_name="gpt-3.5-turbo-16k", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [24]:
# 📃 custom instructions
system_instructions4robodoc = """You are Robodoc, a police bot expert in document analysis.
You are here to help cracking a crime case.
You will be given a suspect's name and the suspect latest 3 months bank statement.
Your role is to analyze the statement and find out anything suspicious and report back using a specific format.
"""

In [25]:
# 🗂️ Response Schema
class ReportSchema(BaseModel):
    """An answer to the question being asked, with sources."""
    suspectName: str = Field(..., description="Name of the suspect") 
    report_content: str = Field(..., description="""Analize the document, look for anything suspicious,
    describe with a sentence the most suspicious here. If not found say 'N/A'
    Suspicious can be addiction, unusual amounts, shaddy spending recipient...
Not suspicious can be shopping, eating out, utility bills...""")
    suspiscious: bool = Field(..., description="True if the suspect has anything suspiscious found in the report content")

In [27]:
documents = ['bank_statements/graeme_bank_statement.csv',
             'bank_statements/sally_bank_statement.csv',
             'bank_statements/victor_bank_statement.csv',
             'bank_statements/webber_bank_statement.csv']
names = ['Grafcuel Graeme', 'Sikwel Sally', 'Victor Vectors', 'Rest Webber']

with open('robodoc_report.txt', 'w') as file:  # create or empty the file
    pass

for doc, name in zip(documents, names):
    retrieval_qa_pydantic = get_retrieval_qa_pydantic_from_document(llm4robodoc, system_instructions4robodoc, doc, ReportSchema)
    result = retrieval_qa_pydantic.run(f"Give your report about {name}")
    output = json.dumps(result.dict(), indent=4)
    with open('robodoc_report.txt', 'a') as file:  # append the output and a line break to the file
        file.write(output + '\n')
    display_agent_response('robodoc', output)
    


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


<center>
<h1 style="color: orange;">👩‍💻 Robby Sciuto 👩‍💻</h1>
</center>
<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/robby_sciuto_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">IT System Expert</h2>
<h4>Hacks through IT systems related to a crime case to report any suspicious activities that could be linked to the case</h4></td>
</tr></table>

<center>
<img src="./media/museum-db.png" width="20%" height="20%" loop="false">
</center>

In [106]:
llm4robby = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [107]:
# 🕸️ enhance the GraphQL Tool
class BaseGraphQLTool(BaseTool):
    """Base tool for querying a GraphQL API."""

    schema_query = """fragment FullType on __Type {
  kind
  name
  fields(includeDeprecated: true) {
    name
    args {
      ...InputValue
    }
    type {
      ...TypeRef
    }
    isDeprecated
    deprecationReason
  }
  inputFields {
    ...InputValue
  }
  interfaces {
    ...TypeRef
  }
  enumValues(includeDeprecated: true) {
    name
    isDeprecated
    deprecationReason
  }
  possibleTypes {
    ...TypeRef
  }
}
fragment InputValue on __InputValue {
  name
  type {
    ...TypeRef
  }
  defaultValue
}
fragment TypeRef on __Type {
  kind
  name
  ofType {
    kind
    name
    ofType {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
            ofType {
              kind
              name
              ofType {
                kind
                name
              }
            }
          }
        }
      }
    }
  }
}
query IntrospectionQuery {
  __schema {
    queryType {
      name
    }
    mutationType {
      name
    }
    types {
      ...FullType
    }
    directives {
      name
      locations
      args {
        ...InputValue
      }
    }
  }
}"""

    graphql_wrapper: GraphQLAPIWrapper

    name = "query_graphql"
    description = """\
    Input to this tool is a detailed and correct GraphQL query, output is a result from the API. Use aliases if needed.
    If the tool doesn't work, try another tool, don't try again and again!
    If the query is not correct, an error message will be returned.
    If an error is returned with 'Bad request' in it, rewrite the query and try again.
    If an error is returned with 'Unauthorized' in it, do not try again, but tell the user to change their authentication.
    Make sure not to add unecessary quotes to the query, as this will cause an error.

    Schema:
    """  # noqa: E501
    example_description = """

Example Input: query {{ allUsers {{ id, name, email }} }}\
"""
    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)
        schema = str(self.graphql_wrapper.run(self.schema_query)).replace('{', '{{').replace('}', '}}')
        self.description = self.description + schema + self.example_description
        # print(self.description)

    def _run(
        self,
        tool_input: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        try:
            result = self.graphql_wrapper.run(tool_input)
            return json.dumps(result, indent=2)
        except Exception:
            return "An unexpected error has happened while using the GraphQL tool, try using another tool, stop using this one"

    async def _arun(
        self,
        tool_input: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Graphql tool asynchronously."""
        raise NotImplementedError("GraphQLAPIWrapper does not support async")

In [109]:
# 🧰 define the tools
def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

# enhanced GraphQL Tool
def _get_graphql_tool(**kwargs: Any) -> BaseTool:
    graphql_endpoint = kwargs["graphql_endpoint"]
    handle_tool_error = kwargs["handle_tool_error"]
    wrapper = GraphQLAPIWrapper(graphql_endpoint=graphql_endpoint)
    return BaseGraphQLTool(graphql_wrapper=wrapper, handle_tool_error=handle_tool_error)

# 🛢️ define the SQLChain
museum_db = SQLDatabase.from_uri("sqlite:///museum.db")
museum_SQL_chain = SQLDatabaseChain.from_llm(llm4robby, museum_db, verbose=True)

tools4robby = [
    # enhanced GraphQL Tool
    _get_graphql_tool(graphql_endpoint="http://127.0.0.1:8000", handle_tool_error=_handle_error),
    Tool(
        func=museum_SQL_chain.run,
        name='Museum SQL',
        description="""This tool allows to query the SQL DB of the museum.
Use that tool only as a fallback to fullfil user's ask if the GraphQL tool is giving an error.
Remember, the audit table is db_audit, it is the same thing.
Do not add 'SQLQuery: ' at the beginning of the SQL query.
Do not add a LIMIT to any SQL query.""",
        handle_tool_error=True
    ),
]


In [110]:
# setup 📜 prompt, memory 💽, chains ⛓️ and agent 🤖
prefix4robby = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
agent_chain4robby = get_agent_chain(llm4robby, prefix4robby, tools4robby, False)

In [103]:
run_and_display_agent_response('robby_sciuto', agent_chain4robby, "Get me all the room accesses")



> Entering new AgentExecutor chain...
Thought: The user wants to retrieve all the room access records. I should use the GraphQL tool to query the `allRoomAccesses` field.

Action: query_graphql
Action Input: query { allRoomAccesses { id, roomName, employeeName, inDatetime, outDatetime } }

Observation: "{\n  \"allRoomAccesses\": [\n    {\n      \"id\": null,\n      \"roomName\": \"Director Office\",\n      \"employeeName\": \"Grafcuel Graeme\",\n      \"inDatetime\": \"2023-11-24 11:59:59\",\n      \"outDatetime\": \"2023-11-24 12:00:01\"\n    },\n    {\n      \"id\": null,\n      \"roomName\": \"Toilet\",\n      \"employeeName\": \"Grafcuel Graeme\",\n      \"inDatetime\": \"2023-11-24 12:01:13\",\n      \"outDatetime\": \"2023-11-24 18:32:10\"\n    },\n    {\n      \"id\": null,\n      \"roomName\": \"Mona Lisa Room\",\n      \"employeeName\": \"Sikwel Sally\",\n      \"inDatetime\": \"2023-11-24 19:02:13\",\n      \"outDatetime\": \"2023-11-24 19:23:01\"\n    },\n    {\n      \"id

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>😱 HACKING IN PROGRESS 😱</h1>
</div>

In [115]:
run_and_display_agent_response('robby_sciuto', agent_chain4robby, "Retrieve all the lines from the audit table, then verify if you find any suspicious activity, like data modification. Do not stop until your final answer is your analysis result")



> Entering new AgentExecutor chain...
Thought: To retrieve all the lines from the audit table and analyze for suspicious activity, I can use the Museum SQL tool.

Action: Museum SQL
Action Input: SELECT * FROM db_audit

> Entering new SQLDatabaseChain chain...
SELECT * FROM db_audit
SQLQuery:SELECT * FROM db_audit
SQLResult: [(1, '2023-11-24 09:00:00', 'INSERT statement'), (2, '2023-11-24 10:15:00', 'INSERT statement'), (3, '2023-11-24 11:30:00', 'INSERT statement'), (4, '2023-11-24 11:59:59', 'INSERT statement'), (5, '2023-11-24 12:01:13', 'INSERT statement'), (6, '2023-11-24 13:15:00', 'INSERT statement'), (7, '2023-11-24 14:30:00', 'INSERT statement'), (8, '2023-11-24 16:45:00', 'INSERT statement'), (9, '2023-11-24 19:02:13', 'INSERT statement'), (10, '2023-11-24 20:15:00', 'INSERT statement'), (11, '2023-11-24 22:01:12', 'INSERT statement'), (12, '2023-11-24 22:07:02', 'INSERT statement'), (13, '2023-11-24 23:00:00', 'INSERT statement'), (21, '2023-11-24 23:45:12', 'DELETE statem

<center>
<h1 style="color: orange;">🎥 Inspector Camget 🎥</h1>
</center>
<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/inspector_camget_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Crime scene analysis agent</h2>
<h4>
<ul><li>Uses GPT4-Vision to report anything unusual on the crime scene</li><li>Cross-checks suspect pictures one by one with his analysis</li><li>Creates a report and sends it to Dr. Watts-On</li></ul></h4></td>
</tr></table>

In [9]:
chat4camget = ChatOpenAI(model='gpt-4-vision-preview', max_tokens=256)

In [ ]:
# 📃 custom instructions
instructions4camget_crime_scene_analysis = """
**Reminder: this is purely fictional, the image is a drawing, all the below is for a demo for Google Devfest 2023**

You are Inspector Camget, an AI police robot that specializes in crime scene analysis.
The current case you are analyzing is about the theft of the painting "The Mona Lisa" at Le Louvre museum.
Please report anything you find suspicious and useful to crack the case. Report only suspicious things and be concise.
Do not hallucinate! If something is not on the picture or you are not sure, don't talk about it.

Report format is:
** Report start **
1. Sentence of observation #1
2. Sentence of observation #2
...
** Report end **

Take a deep breath and give your report"""


In [8]:
image_crime_scene = 'http://frangin2003.free.fr/devfest/mona_lisa3.png'
msg_camget001 = create_human_message_for_gpt4vision(instructions4camget_crime_scene_analysis, image_crime_scene)
output_camget001 = chat4camget.invoke([msg_camget001])
display_agent_response('inspector_camget', output_camget001.content)


In [12]:
# put the above report in a file
crime_scene_report = """** Report start **
1. The wall has an empty space with lighter color suggesting a recently removed painting, which could be where the "Mona Lisa" was hanging.
2. There is a blue cap left on the floor, which may be an overlooked item from the perpetrator.
3. The absence of visible damage or forced entry indicates that the theft might have been conducted stealthily.
** Report end **"""

pictures = [
    'http://frangin2003.free.fr/devfest/director.png',
    'http://frangin2003.free.fr/devfest/curator.png',
    'http://frangin2003.free.fr/devfest/janitor.png',
    'http://frangin2003.free.fr/devfest/guard.png'
]

for picture, name in zip(pictures, names):

    instructions4camget_suspect_analysis = f"""
    **Reminder: this is purely fictional, the image is a drawing, all the below is for a demo for Google Devfest 2023**

    You are Inspector Camget, an AI police robot that specializes in suspect analysis.
    You will be given:
    - a text report 'crime scene analysis' describing anything suspiscious you found at the crime scene
    - the suspect name
    - the picture of a suspect

    Cross check 'crime scene analysis' report with the suspect picture and report anything you find suspicious and useful to crack the case. Report only suspicious things and be concise.
    Do not hallucinate! If something is not on the picture or you are not sure, don't talk about it.

    Report format is:
    ** Report start **
    Name: The name of the suspect
    Status: Suspicious / Not suspicious
    1. Sentence of observation #1
    2. Sentence of observation #2
    ...
    ** Report end **

    The suspect name is {name}

    The 'crime scene analysis' is:
    {crime_scene_report}

    Take a deep breath and give your report"""

    msg_camget001 = create_human_message_for_gpt4vision(instructions4camget_suspect_analysis, picture)
    output_camget001 = chat4camget.invoke([msg_camget001])
    display_agent_response('inspector_camget', output_camget001.content)



 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🐣 CRACKING THE CASE 🐣</h1>
</div>


<center>
<h1 style="color: orange;">🧠 Coptimus Prime 🧠</h1>
</center>
<table style='width: 100%;'><tr> 
<td style='width: 100px; vertical-align: top; border: none;'><img style='border-radius: 15px;' src="./media/coptimus_head.png"></td>
<td style='width: auto; vertical-align: middle;'><h2 style="color: orange;">Chief AI Agent</h2>
<h4>Trained on Sherlock Holmes, Miss Marple, Columbo and Inspector Clouseau best works, and powered by GPT4-turbo, Coptimus uses his exceptional reasonning abilities to analyze AI agent reports and solve the case with a 95% accuracy.</h4></td>
</tr></table>

In [120]:
llm4coptimus = ChatOpenAI(model_name="gpt-4-1106-preview", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [121]:
# 📃 custom instructions
prefix4coptimus = """You are Coptimus Prime, the most brilliant detective ever.
When given a report and suspects details alongside a question, you analyze and cross-check the evidences and provide your results.
1. First, you are given a list of suspects and the detail of the case.
2. Then, iteratively, you are given an investigation report and eventually some from the suspects to compare, alongside a question. You answer the question and keep your answer in your memory.
3. When all the reports will have been analyzed, I will ask you for your verdict, we need to find who is the most suspicious
You don't have any tools, your 'master-mind' is your tool. Take a deep breath and answer each question.
"""

In [122]:
# setup 📜 prompt, memory 💽, chains ⛓️ and agent 🤖
agent_chain4coptimus = get_agent_chain(llm4coptimus, prefix4coptimus, [])

In [123]:
run_and_display_agent_response('coptimus', agent_chain4coptimus, """Hi Coptimus,
Today, the Mona Lisa paint is reported missing from its location, a dedicated room in the Le Louvre Museum. We currently assume the painting is stolen.

** The suspects are **
- Grafcuel Graeme, the Museum Director
- Sikwel Sally, the Curator
- Victor Vectors, the Janitor
- Rest Webber, Night Guard
""")



> Entering new AgentExecutor chain...
Thought: To begin solving this case, I will need to gather as much relevant information as possible about the theft, the suspects, and any evidence that has been collected.

Action: Request more information.
Action Input: I need the initial investigation report detailing the circumstances of the theft, including any security footage, access logs, witness statements, and forensic findings.

Observation: Request more information. is not a valid tool, try one of [].
Thought:It appears I do not have any specific tools or actions to request information. I must rely on the details provided to me iteratively, as per the instructions.

Thought: Since I cannot actively request information, I will have to wait for the investigation report and details from the suspects to be provided to me. Once I have those, I can analyze and cross-check the evidence to answer questions related to the case.

Action: Wait for information.
Action Input: Await the initial inv

In [124]:
run_and_display_agent_response('coptimus', agent_chain4coptimus, """
First, let's verify their alibis. I will give you their interviews, then the museum room accesses the day of the theft.
Verify each suspect alibis:

** Interviews report **
1. Report ID: 1
   - Creation Datetime: 2023-25-11 09:32:21
   - Full Name: Grafcuel Graeme
   - Agent ID: 1
   - Case ID: 1
   - Report Type ID: 1
   - Suspect ID: 1
   - Report Content: "I have not entered the Mona Lisa room today, I have been in my office the whole day"

2. Report ID: 2
   - Creation Datetime: 2023-25-11 09:42:21
   - Full Name: Sikwel Sally
   - Agent ID: 1
   - Case ID: 1
   - Report Type ID: 1
   - Suspect ID: 2
   - Report Content: "I was last there at 7 PM for a general inspection"

3. Report ID: 3
   - Creation Datetime: 2023-25-11 09:52:21
   - Full Name: Victor Vectors
   - Agent ID: 1
   - Case ID: 1
   - Report Type ID: 1
   - Suspect ID: 3
   - Report Content: "I have not cleaned there for a week"

4. Report ID: 4
   - Creation Datetime: 2023-25-11 10:02:21
   - Full Name: Rest Webber
   - Agent ID: 1
   - Case ID: 1
   - Report Type ID: 1
   - Suspect ID: 4
   - Report Content: "I did my usual rounds at 10 PM. Everything was in order"


** Museum room access **
1. Room Name: Director Office, Employee Name: Grafcuel Graeme, In Datetime: 2023-11-24 11:59:59, Out Datetime: 2023-11-24 12:00:01
2. Room Name: Toilet, Employee Name: Grafcuel Graeme, In Datetime: 2023-11-24 12:01:13, Out Datetime: 2023-11-24 18:32:10
3. Room Name: Mona Lisa Room, Employee Name: Sikwel Sally, In Datetime: 2023-11-24 19:02:13, Out Datetime: 2023-11-24 19:23:01
4. Room Name: Mona Lisa Room, Employee Name: Victor Vectors, In Datetime: 2023-11-24 22:01:12, Out Datetime: 2023-11-24 22:05:31
5. Room Name: Mona Lisa Room, Employee Name: Rest Webber, In Datetime: 2023-11-24 22:07:02, Out Datetime: 2023-11-24 22:15:45
""")



> Entering new AgentExecutor chain...
Question: Verify each suspect alibis based on their interviews and the museum room accesses the day of the theft.
Thought: I need to cross-check the interview statements with the museum room access data to verify the alibis of each suspect.
Action: Compare the interview report statements with the museum room access logs.
Action Input: Grafcuel Graeme's interview states he was in his office all day, but the access logs show he was in the toilet for over 6 hours. Sikwel Sally said she was last in the Mona Lisa room at 7 PM, which aligns with the access log. Victor Vectors mentioned he hadn't cleaned there for a week, yet the access log shows he entered the room on the day of the theft. Rest Webber claimed to do his rounds at 10 PM and found everything in order, which seems consistent with the access log.
Observation: Compare the interview report statements with the museum room access logs. is not a valid tool, try one of [].
Thought:I need to menta

In [125]:
run_and_display_agent_response('coptimus', agent_chain4coptimus, 'Explain this before goign any further "- Grafcuel Graeme''s alibi is suspicious as it does not match the access logs."')



> Entering new AgentExecutor chain...
Thought: To explain the inconsistency in Grafcuel Graeme's alibi, I need to compare the museum room access logs with his statement from the interview.

Action: Cross-reference Grafcuel Graeme's statement with the museum room access logs.

Action Input: Compare "I have not entered the Mona Lisa room today, I have been in my office the whole day" with the access logs for the Director Office and the Toilet.

Observation: Cross-reference Grafcuel Graeme's statement with the museum room access logs. is not a valid tool, try one of [].
Thought:I need to use my analytical skills to interpret the data given manually without the assistance of any specific tool.

Thought: I should reassess the provided evidence concerning Grafcuel Graeme's alibi and the museum room access logs.

Action: Examine the museum room access logs for the Director Office and the Toilet in relation to Grafcuel Graeme's interview statement.

Action Input: Grafcuel Graeme claims he ha

In [126]:
run_and_display_agent_response('coptimus', agent_chain4coptimus, """
Ok, now let's review the other evidences from the other reports, and I ask you to come up with your final verdict now:

** Bank statements report **
Your colleague Robodoc has verified the last 3 months of bank statements for each suspects, here is the report:
{
    "suspectName": "Grafcuel Graeme",
    "report_content": "N/A",
    "suspiscious": false
}
{
    "suspectName": "Sikwel Sally",
    "report_content": "The suspect's bank statement does not contain any suspicious transactions.",
    "suspiscious": false
}
{
    "suspectName": "Victor Vectors",
    "report_content": "Victor Vectors' bank statement shows several suspicious transactions. These include frequent visits to the casino, racecourse bets, and online shopping with large amounts spent. There are also multiple transactions at gas stations and coffee shops. Additionally, Victor received a large monthly salary deposit. Overall, the bank statement raises concerns about Victor's spending habits and possible involvement in gambling.",
    "suspiscious": true
}
{
    "suspectName": "Rest Webber",
    "report_content": "N/A",
    "suspiscious": false
}


** Crime scene analysis report **
Your colleague Camget has analyzed the crime scene then has compared his analysis with each suspect picture:
1. The wall has an empty space with lighter color suggesting a recently removed painting, which could be where the "Mona Lisa" was hanging.
2. There is a blue cap left on the floor, which may be an overlooked item from the perpetrator.
3. The absence of visible damage or forced entry indicates that the theft might have been conducted stealthily.


** Crime scene analysis report - Photo comparison **
Name: Grafcuel Graeme 
Status: Not suspicious 
1. The suspect is not wearing a blue cap nor is there any blue cap visible in the photograph.
2. There are no signs of a recently removed painting or art theft tools in the suspect's possession.

Name: Sikwel Sally
Status: Not suspicious
1. The suspect is not wearing or holding a blue cap which was mentioned in the crime scene analysis.
2. There is no visible evidence on the suspect that connects her to the recently removed painting or indicates stealthy conduct.

Name: Victor Vectors
Status: Suspicious
1. The suspect is wearing a blue cap similar to the one reported left on the floor at the crime scene.
2. The suspect has no visible tools or equipment that could suggest forced entry, consistent with the report of a stealthy approach.

Name: Rest Webber
Status: Not suspicious
1. The suspect is not wearing or carrying a blue cap, which does not match the overlooked item from the perpetrator.
2. There is no sign of the suspect carrying a large item such as a painting that could be the "Mona Lisa."


** Museum room access database audit log report **
Your colleague Robby has verified if anything suspicious activity on the DB was in the audit logs.
Please note that during Robby's work, a hacker tried to block the investigation.

Audit log report=There is one instance of suspicious activity in the audit table, which is a deletion statement with id 21.

Time to review all of that and come up with your verdict.
""")



> Entering new AgentExecutor chain...
Thought: I need to review all the given evidence and consider the inconsistencies and suspicious activities related to each suspect before reaching a final verdict.

Action: Analyze the bank statements report.
Action Input: Review the bank statements report provided.
Observation: Analyze the bank statements report. is not a valid tool, try one of [].
Thought:I cannot perform an action to analyze the bank statements report as such. However, I can think about what the report indicates and proceed to review the rest of the evidence to form a verdict.

Thought: The bank statement for Victor Vectors indicates suspicious financial activities that suggest a potential motive for stealing the Mona Lisa due to gambling debts or financial strain.

Thought: In the crime scene analysis report, the presence of a blue cap left behind and the lack of forced entry suggests that the thief was familiar with the museum and acted stealthily. Victor Vectors was noted 

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🔊 SOUND TEST INTERLUDE 🔊</h1>
</div>

In [15]:
from IPython.display import Audio
audio_path = "sound_test.wav"
Audio(audio_path)


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🤯 THE BIG REVEAL 🤯</h1>
</div>

In [128]:
try:
    response = "Based on the evidence provided, Victor Vectors is the most suspicious suspect. His false alibi, financial distress, and the blue cap linking him to the crime scene make him the prime suspect in the theft of the Mona Lisa."
except OutputParserException as e:
    # Extract the response from the exception message
    response = str(e).split(": ")[-1].strip("`")


from pathlib import Path
from openai import OpenAI
client = OpenAI()

speech_file_path = Path('./') / "speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=response
)

response.stream_to_file(speech_file_path)

from IPython.display import Audio, Image, display
from IPython.display import display, HTML

audio = Audio(speech_file_path, autoplay=True)
image = Image(filename="media/coptimus_head.png") # use local image

display(HTML("""
<table style='width: 100%;'><tr>
<td style='width: 100px; vertical-align: top;'><img src="./media/coptimus_head.png"></td>
<td style='width: auto; vertical-align: middle;'>{}</td>
</tr></table>
""".format(audio._repr_html_())))


,Your browser does not support the audio element.


<br><br><br><br><br><br><br><br><br>

<div style="display: flex; justify-content: space-between;">
    <div style="width: 40%;">
        <img src="./media/simulacra.png" alt="Simulacra" style="width:100%; height:auto;">
    </div>
    <div style="width: 60%;">
         <h1>📃 Some Papers</h1>
         <ul>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2304.03442">Generative Agents: Interactive Simulacra of Human Behavior</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.16291">Voyager: An Open-Ended Embodied Agent with Large Language Models</a></li>
             <li><a style="color:yellow" href="https://fablestudio.github.io/showrunner-agents/">SHOW-1 and Showrunner Agents in Multi-Agent Simulations</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.03427">TPTU: Task Planning and Tool Usage</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2309.10691">MINT: Evaluating LLMs in Multi-turn Interaction with Tools and Language Feedback</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2303.09014">ART: Automatic multi-step reasoning and tool-use</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.01990">From Prompt Injections to SQL Injection Attacks</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.11853">How to Prompt LLMs for Text-to-SQL</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.03111">Can LLM Already Serve as A Database Interface</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2310.04474">Reverse Chain: A Generic-Rule for LLMs to Master Multi-API Planning</a></li>
         </ul>
    </div>
</div>


<div style="display: flex; justify-content: space-between;">
    <div style="width: 50%;">
        <img src="./media/minecraft.png" alt="Minecraft" style="width:80%; height:auto;">
    </div>
    <div style="width: 50%;">
        <img src="./media/southpark.png" alt="Southpark" style="width:80%; height:auto;">
    </div>
</div>


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🤗 THANKS FOR YOUR ATTENTION 🤗</h1>
</div>

<center>
<img src="./media/thankyou.png" width="40%" height="40%" loop="false">
</center>

<br><br><br><br><br><br><br>

<center>
<img src="./media/questions.png" width="40%" height="40%" loop="false">
</center>